In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Bidirectional

import tensorflow as tf
import keras.backend as K

Using TensorFlow backend.


In [2]:
# read data (for now, sell_prices & calendar are not used)

data_dir = 'data/'

train_sales = pd.read_csv(data_dir + 'sales_train_validation.csv')
#sell_prices = pd.read_csv(data_dir + 'sell_prices.csv')
#calendar = pd.read_csv(data_dir + 'calendar.csv')
submission_file = pd.read_csv(data_dir + 'sample_submission.csv')

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
train_sales = reduce_mem_usage(train_sales)

Mem. usage decreased to 95.00 Mb (78.7% reduction)


In [5]:
# create training data, for now it only contains the sales and no extra features
sales = train_sales.drop(["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"], axis=1).T

# normalize
scaler = MinMaxScaler()
scaler.fit(sales)
sales = scaler.transform(sales)
sales = pd.DataFrame(sales)

In [6]:
# create X and y

timesteps = 28
prediction_steps = 1
len_window = timesteps + prediction_steps

nr_training_days = sales.shape[0]
nr_sets = nr_training_days - len_window + 1

base, predictions = [], []

for i in range(nr_sets):
    samples = sales.iloc[i:i+timesteps]
    pred = sales.iloc[i+timesteps]
    base.append(samples.to_numpy())
    predictions.append(pred.to_numpy())
    
X = np.array(base)
y = np.array(predictions)

del base, predictions

In [7]:
def root_mean_squared_scaled_error(y_true,y_pred):
    sample_length = 28
    forecasting_horizon = 1
    upper_bound = sample_length + forecasting_horizon + 1
    lower_bound = sample_length + 1 - 1
    numerator =  K.sum(K.square(y_true[lower_bound:upper_bound]-y_pred[lower_bound:upper_bound]))
    lower_bound = 2 - 1
    denominator = (1/(sample_length - 1 )) * K.sum(K.square(y_true[lower_bound:sample_length] - y_true[lower_bound-1:sample_length-1]))
    value_to_be_sqrt = (1/forecasting_horizon) * (numerator/denominator)
    result = K.sqrt(value_to_be_sqrt)
    return result

In [8]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor = 0.1, patience = 2 , verbose = 1, mode= 'min', min_lr = 0.000001)





In [18]:
# create model

n_features = X.shape[2]

model = Sequential()
model.add(Bidirectional(LSTM(20, return_sequences=True, input_shape=(timesteps, n_features))))
model.add(Bidirectional(LSTM(10)))
model.add(Dense(30490))
model.compile(optimizer='adam', loss=root_mean_squared_scaled_error)

In [19]:
# train model

model.fit(X, y, batch_size=32, epochs=10, callbacks=[reduce_lr], verbose=1)

Epoch 1/10
1885/1885 [==============================] - 91s 48ms/step - loss: 0.7302
Epoch 2/10
1885/1885 [==============================] - 80s 43ms/step - loss: 0.7143
Epoch 3/10
1885/1885 [==============================] - 82s 43ms/step - loss: 0.7135
Epoch 4/10
1885/1885 [==============================] - 82s 44ms/step - loss: 0.6901
Epoch 5/10
1885/1885 [==============================] - 83s 44ms/step - loss: 0.6916
Epoch 6/10
1885/1885 [==============================] - 80s 43ms/step - loss: 0.6850
Epoch 7/10
1885/1885 [==============================] - 80s 42ms/step - loss: 0.6899
Epoch 8/10
1885/1885 [==============================] - 80s 42ms/step - loss: 0.6870

Epoch 00008: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 9/10
1885/1885 [==============================] - 79s 42ms/step - loss: 0.6896
Epoch 10/10
1885/1885 [==============================] - 78s 41ms/step - loss: 0.6739


In [20]:
# get predictions

for i in range(28):    
    # get input for prediction by selecting last 28 days from sales
    X_pred = []
    X_pred.append(sales.iloc[-timesteps:].to_numpy())
    X_pred = np.array(X_pred)
    
    # get prediction
    prediction = model.predict(X_pred)
    
    # add prediction to sales so that it can be used for next prediction
    sales.loc[sales.shape[0]] = prediction[0]
    
predictions = sales.iloc[-28:]
predictions = scaler.inverse_transform(predictions)
predictions = np.round(np.abs(predictions))
predictions = pd.DataFrame(predictions).T

In [21]:
# create submission file
predictions_copy = predictions
final_submission = pd.concat([predictions, predictions_copy])
final_submission.reset_index(drop=True, inplace=True)
final_submission = final_submission.astype(int)
final_submission.insert(0, 'id', submission_file['id'])
final_submission.columns = ['id'] + [f"F{i}" for i in range(1, 29)]

final_submission.to_csv('submission_msqce_lronplateau.csv', index=False)